<a href="https://colab.research.google.com/github/LeonVillanueva/CoLab/blob/master/Google_CoLab_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -q tensorflow==2.0.0-beta1

In [0]:
%%capture
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [0]:
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout, GlobalMaxPooling2D, GaussianNoise, BatchNormalization, MaxPooling2D, SimpleRNN, GRU, LSTM, GlobalMaxPooling1D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [0]:
from scipy import stats
import math
import seaborn as sns
import re

*https://www.gutenberg.org/files/4357/4357-h/4357-h.htm*

In [0]:
text = ['She flew down the stairs, found the basket and returned with it to the attic.',
        'Then she sat down before the brass-studded box and began trying one key after another in the curious old lock.',
        'Some were too large, but most were too small.']

max_vocab_size = 20000

tokenizer = Tokenizer (num_words = max_vocab_size)
tokenizer.fit_on_texts (text)
sequences = tokenizer.texts_to_sequences (text)

In [0]:
data = pad_sequences (sequences, padding='post')

In [0]:
df = pd.read_csv ('wiki_movie_plots_deduped.csv')

In [53]:
df.columns.values

array(['Release Year', 'Title', 'Origin/Ethnicity', 'Director', 'Cast',
       'Genre', 'Wiki Page', 'Plot'], dtype=object)

In [0]:
df = df[(df['Origin/Ethnicity'] == 'American') & (df['Genre'] != 'unknown')]

In [56]:
genres.value_counts()[:10]

drama              3549
comedy             3059
western             855
horror              782
action              495
thriller            481
adventure           442
crime drama         414
musical             386
science fiction     355
Name: Genre, dtype: int64